<a href="https://www.kaggle.com/code/dewizzz/lalalayeyeyelulusfixed?scriptVersionId=142769953" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/dewizzz/lalalayeyeyelulusfixed?scriptVersionId=138655296" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1 style="background-color:#F0E68C;font-family:Halvetica;font-size:550%;text-align:center;border-radius: 15px 10px;padding: 5px"><b>Deteksi Autisme</b></h1>

In [ ]:
import os
import time
import torch
import seaborn as sn
import pandas as pd
from PIL import Image
from torch import nn, optim
import matplotlib.pyplot as plt
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from tqdm.notebook import tqdm
import torch.nn.functional as F

# **Load Data**

In [ ]:
# Path ke folder kelas 1
folder_kelas_1 = '/kaggle/input/data-skripsi/Data Skripsi/Kelas Autisme'

# Path ke folder kelas 0
folder_kelas_0 = '/kaggle/input/data-skripsi/Data Skripsi/Kelas TC'

# Inisialisasi dictionary untuk menyimpan label
label_gambar = {}

# Melabeli gambar-gambar dalam folder kelas 1
for file in os.listdir(folder_kelas_1):
    if file.endswith('.gif'):
        label_gambar[file] = 1  # Label kelas 1

# Melabeli gambar-gambar dalam folder kelas 0
for file in os.listdir(folder_kelas_0):
    if file.endswith('.gif'):
        label_gambar[file] = 0  # Label kelas 0

In [ ]:
# Mencetak hasil labeling untuk 5 gambar dari setiap kelas
count_kelas_0 = 0
count_kelas_1 = 0

for file, label in label_gambar.items():
    if label == 0 and count_kelas_0 < 5:
        print(f"File: {file}, Label: {label}")
        count_kelas_0 += 1
    elif label == 1 and count_kelas_1 < 5:
        print(f"File: {file}, Label: {label}")
        count_kelas_1 += 1
    
    if count_kelas_0 == 5 and count_kelas_1 == 5:
        break

In [ ]:
# Transformasi untuk data gambar GIF
class GIFToTensor(object):
    def __call__(self, pic):
        if pic.mode != 'RGB':
            pic = pic.convert('RGB')
        return transforms.ToTensor()(pic)

# Transformasi data
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(201),
    transforms.Resize(224),
    GIFToTensor(),
])

# Proses transformasi dan penyimpanan data
data = []

for file, label in label_gambar.items():
    # Baca gambar dan aplikasikan transformasi
    img_path = os.path.join(folder_kelas_1 if label == 1 else folder_kelas_0, file)
    image = Image.open(img_path)
    transformed_image = transform(image)
    
    # Simpan data dan label ke dalam list
    data.append((transformed_image, label))

# **Split Data**

In [ ]:
# Bagi data menjadi set pelatihan dan set pengujian
train_data, test_data, train_labels, test_labels = train_test_split(
    [item[0] for item in data],  # Data gambar
    [item[1] for item in data],  # Label
    test_size=0.3,  # Proporsi set pengujian
    random_state=42  # Seed untuk reproduktibilitas
)

# Cetak jumlah data dalam set pelatihan dan set pengujian
print(f"Jumlah data pelatihan: {len(train_data)}")
print(f"Jumlah data pengujian: {len(test_data)}")

In [ ]:
# Menampilkan 5 gambar dari set pelatihan
fig, axes = plt.subplots(1, 5, figsize=(10, 4))

for i, ax in enumerate(axes):
    img = train_data[i].permute(1, 2, 0)  # Mengubah dimensi tensor menjadi (H, W, C)
    label = train_labels[i]
    ax.imshow(img)
    ax.set_title(f"Label: {label}")
    ax.axis('off')

plt.tight_layout()
plt.show()

# **Split Data Train - Val**

# Memisahkan data pelatihan menjadi data validasi dan data pelatihan baru
train_data, val_data, train_labels, val_labels = train_test_split(
    train_data,  # Data pelatihan yang sudah ada
    train_labels,  # Label pelatihan yang sudah ada
    test_size=0.3,  # Proporsi data validasi
    random_state=42  # Seed untuk reproduktibilitas
)

# Cetak jumlah data dalam set pelatihan dan set pengujian
print(f"Jumlah data pelatihan: {len(train_data)}")
print(f"Jumlah data pengujian: {len(val_data)}")

# Perbaikan pixel (buat jaga-jaga)

In [ ]:
train_dataset = list(zip(train_data, train_labels))
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

test_dataset = list(zip(test_data, test_labels))
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Menginisialisasi model pretrained
model = models.vgg16(pretrained=True)

# Menonaktifkan penghitungan gradien untuk parameter model pretrained
for param in model.parameters():
    param.requires_grad = False

# Mengubah ukuran input pada lapisan konvolusi pertama
model.features[0] = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)

# Mengganti layer terakhir (fully connected) pada model dengan layer baru
num_classes = 2  # Jumlah kelas
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

In [ ]:
print(model)

In [ ]:
# Menggunakan perangkat GPU jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Mendifinisikan fungsi loss dan optimizer
criterion = nn.CrossEntropyLoss()
learning_rate = 1e-5
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# **Train Model**

In [ ]:
from sklearn.model_selection import KFold

train_loss = []
train_accuracy = []
valid_loss = []
valid_accuracy = []

def train(model, device, train_loader, optimizer, epoch, train_indices, log_interval=10000):
    model.train()
    tk0 = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    running_loss = 0.0
    running_corrects = 0
    for batch_idx, (data, target) in enumerate(tk0):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        logits = output
        output = F.softmax(output, dim=1)
        _, preds = torch.max(output, dim=1)
        loss = criterion(logits, target)
        loss.backward()
        optimizer.step()
        counter += 1
        tk0.set_postfix(loss=(loss.item()*data.size(0) / (counter * train_loader.batch_size)))
        running_loss += loss.item()
        running_corrects += torch.sum(preds == target).item()
        
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = running_corrects / len(train_indices)  # Use len(train_indices) instead of len(train_loader.dataset)
    train_loss.append(epoch_loss)
    train_accuracy.append(epoch_acc)
    print(epoch_loss)
    print(epoch_acc)


def val(model, device, val_loader, val_indices):
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            logits = output
            val_loss += criterion(logits, target).item() # sum up batch loss
            pred_prob = torch.softmax(logits, dim=1) # apply softmax to output
            pred_class = torch.argmax(pred_prob, dim=1) # get the index of the max probability
            correct += pred_class.eq(target.view_as(pred_class)).sum().item()
    val_acc= correct/len(val_indices)  # Use len(val_indices) instead of len(val_loader.dataset)
    val_loss /= len(val_loader)
    valid_loss.append(val_loss)
    valid_accuracy.append(val_acc)
    print(val_loss)
    print(val_acc)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(val_indices),
        100. * correct / len(val_indices)))  # Use len(val_indices) instead of len(val_loader.dataset)

In [ ]:
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
epochs = 40
batch_size = 32

train_loss_per_fold = []
train_accuracy_per_fold = []
valid_loss_per_fold = []
valid_accuracy_per_fold = []

start_time = time.time()

for fold, (train_indices, val_indices) in enumerate(kf.split(train_dataset)):
    print(f"Fold {fold + 1}/{kf.get_n_splits()}")  # Using kf.get_n_splits() to get the number of splits

    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=SubsetRandomSampler(train_indices))
    val_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=SubsetRandomSampler(val_indices))

    for epoch in range(epochs):
        train(model, device, train_loader, optimizer, epoch, train_indices)
        val(model, device, val_loader, val_indices)

    # Store the outputs for each fold
    train_loss_per_fold.append(train_loss)
    train_accuracy_per_fold.append(train_accuracy)
    valid_loss_per_fold.append(valid_loss)
    valid_accuracy_per_fold.append(valid_accuracy)

print("--- %s seconds ---" % (time.time() - start_time))


import time

start_time = time.time()

num_epoch = 100
for epoch in range(1, num_epoch + 1):
    train(model, device, train_loader, optimizer, epoch, train_indices, val_indices)
    val(model, device, val_loader, val_indices)
    
    print("Epoch:", epoch)
    

print("--- %s seconds ---" % (time.time() - start_time))


# **Evaluasi Model**

In [ ]:
# Evaluasi model pada data pengujian
model.eval()
test_loss = 0.0
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        images, labels = batch[0].to(device), batch[1].to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

test_accuracy = total_correct / total_samples
test_loss /= len(test_loader)

# Mencetak loss dan akurasi pada data pengujian
print(f"Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy*100:.2f}%")



In [ ]:
dictionary = {
    'train_loss':train_loss,
    'train_accuracy':train_accuracy,
    'val_loss':valid_loss,
    'val_accuracy':valid_accuracy
}
metrics = pd.DataFrame(dictionary)
metrics

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('Model Metrics Plots')

ax1.plot(metrics[['train_loss', 'val_loss']])
ax1.legend(['train_loss', 'val_loss'], loc="upper right")

ax2.plot(metrics[['train_accuracy', 'val_accuracy']])
ax2.legend(['train_accuracy', 'val_accuracy'], loc="lower right");

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = []
y_true = []

for data, labels in test_loader:
        data = data.to(torch.device('cuda'))
        output = model(data) # Feed Network
        
        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output) # Save Prediction
        
        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # Save Truth

classes = ("Normal", "Autisme")

In [ ]:
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, fmt="d",annot=True)
print(classification_report(y_true, y_pred))
